# DTSA 5511: GAN Project

## Project Overview

For this project, I will be competiting in the "I'm Something of a Painter Myself" competition from Kaggle. The objective will be to create a deep learning model that can be trained on images of paintings and then create new images of paintings on its own. The trained model will take ordinary pictures and turn them into "Monet" style paintings. My initial plan is to construct a GAN model using layers from the Keras library, and make adjustments as time permits.

In [1]:
#Importing Libraries & Packages
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
import os
import zipfile

## Monet Images Data Set

Here...

## Data Cleaning

Here...

In [3]:
#Loading in Processed Images
def load_images(dir):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        dir,
        image_size=(256, 256),
        batch_size=1,
        label_mode=None
    )
    return dataset.map(lambda x: (x - 128) / 128).repeat()

monet_dataset = load_images('/Users/jschlangen/Desktop/gan-getting-started/monet_jpg')
photo_dataset = load_images('/Users/jschlangen/Desktop/gan-getting-started/photo_jpg')

Found 300 files.
Found 7038 files.


## Exploratory Data Analysis

Here...

In [2]:
#ADD EDA HERE

## Model Architecture

Here...

In [4]:
#Construct Image Generator
def build_generator():
    inputs = layers.Input(shape=(256, 256, 3))
    x = layers.Conv2D(64, kernel_size=7, padding='same', activation='relu')(inputs)
    x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    outputs = layers.Conv2D(3, kernel_size=7, padding='same', activation='tanh')(x)
    return Model(inputs, outputs)

my_generator = build_generator()
optimizer = tf.keras.optimizers.Adam()

In [5]:
#Contruct and Train GAN Model
@tf.function
def train(x, y):
    with tf.GradientTape() as tape:
        yp = my_generator(x)
        loss = tf.reduce_mean(tf.square(yp - y))

    grads = tape.gradient(loss, my_generator.trainable_variables)

    optimizer.apply_gradients(zip(grads, my_generator.trainable_variables))

epochs = 5
steps = 350
for epoch in range(epochs):
    for step in range(steps):
        x = next(iter(photo_dataset))
        y = next(iter(monet_dataset))
        train(x, y)

## Results & Analysis

Here...

Add Submission image?

In [ ]:
#Generate Images
output_loc = 'generated_images'
os.makedirs(output_loc, exist_ok=True)

count = 0
max_images = 10000

for x in photo_dataset:
    while count < max_images:
        image = my_generator(x)[0]
        image = ((image.numpy() + 1) * 255 / 2).astype(int)
        
        filename = os.path.join(output_loc, 'generated_' + str(count) + '.jpg')
        tf.keras.preprocessing.image.save_img(filename, image)
        
        count += 1

In [ ]:
#Create .zip file for Kaggle Submission
with zipfile.ZipFile('images.zip', 'w') as zipf:
    for root, _, files in os.walk(output_loc):
        for file in files:
            zipf.write(os.path.join(root, file), file)

## Conclusion

Here...